# Stolen Bases: A Matter of Skill or Guts?

<img src='https://img.mlbstatic.com/mlb-images/image/upload/t_2x1/t_w1536/mlb/yblyorebwvue0kwl7y0b.jpg' width='600' align='center'/>

# Business Understanding

MLB saw an increase in stolen bases in 2023. An MLB team wants to increase viewer retention rates and increase the fan base by adding more action into the game, and they've decided on joining the fun with stolen bases. In this project, I will be advising the MLB team on **how to increase SB stats** for their players. Is it a matter of skill or guts? While this MLB team wants to increase SB stats, they do not want it to jeopardize their wins.

I investigate the following question:
- What is the number of **stolen base attempts** that would **increase action** in the game ***without*** greatly **increasing the chances of a loss**?

# Data Understanding

This data was extracted from a [custom leaderboard](https://baseballsavant.mlb.com/leaderboard/custom?year=2023&type=batter&filter=&min=q&selections=r_total_caught_stealing%2Cr_total_stolen_base%2Cn_bolts%2Csprint_speed&chart=true&x=r_total_caught_stealing&y=r_total_caught_stealing&r=no&chartType=beeswarm&sort=r_total_stolen_base&sortDir=desc) I created on **Baseball Savant**. The data includes **CS** (caught stealing), **SB** (stolen base), **Bolts** and **Sprint Speed** from **133 players in the MLB** (Major League Baseball) from 2023. This data has been collected from **Statcast**, "a state-of-the-art tracking technology, capable of measuring previously unquantifiable aspects of the game."([Baseball Savant](https://baseballsavant.mlb.com/about#:~:text=Where%20is%20the%20data%20from,probable%20pitchers%20for%20upcoming%20days.))

I began with importing the necessary libraries for my data preparation and exploratory data analysis. These libraries are for data manipulation and data visualization.

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

In [3]:
player_data_df = pd.read_csv('stats.csv')

## Data Preparation

Since the data was extracted from a custom leaderboard I created on [Baseball Savant](https://baseballsavant.mlb.com/), there was minimal cleaning that needed to be done. 

First, I previewed the first 5 entries of the table. This gives me a general idea on how I want to clean and handle the data. I notice:
1. I want to **clean the column names**.
2. I already see **NaNs for n_bolts**. I will have to decide how to handle that.
3. Team names are not included in the dataset. I could add team rosters to a variable and feature engineer a column for teams, but I am more **focused on specific players** stealing bases, not on overall team statistics.
4. On Baseball Savant, my customer leaderboard was **sorted** by descending stolen bases but the data did *not* import that way. I will keep that in mind as I proceed with my analysis.

In [5]:
# View the first 5 entries of the table
player_data_df.head()

,"last_name, first_name",player_id,year,r_total_caught_stealing,r_total_stolen_base,n_bolts,sprint_speed
0,"Candelario, Jeimer",600869,2023,1,8,NaN,27.5
1,"McMahon, Ryan",641857,2023,5,5,NaN,25.8
2,"Muncy, Max",571970,2023,2,1,NaN,26.9
3,"Soler, Jorge",624585,2023,0,1,NaN,26.6
4,"Edman, Tommy",669242,2023,4,27,7.0,28.8


Below I noticed as I viewed the overall makeup of the data that n_bolts is the only column with nulls. Thats 

In [7]:
# View the overall shape, dtypes and null counts for each column
player_data_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 133 entries, 0 to 132
Data columns (total 7 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   last_name, first_name    133 non-null    object 
 1   player_id                133 non-null    int64  
 2   year                     133 non-null    int64  
 3   r_total_caught_stealing  133 non-null    int64  
 4   r_total_stolen_base      133 non-null    int64  
 5   n_bolts                  50 non-null     float64
 6   sprint_speed             133 non-null    float64
dtypes: float64(2), int64(4), object(1)
memory usage: 7.4+ KB


Below shows the summary statistics of the dataset. Although all of the data below is numerical, player_id and year are categorical data, so those can be ignored. r_total_caught_stealing, r_total_stolen_base, n_bolts, and sprint_speed, however, give me a general overview of the values I expect to see in the data set.

In [8]:
# View summary statistics of the dataset.
player_data_df.describe()

,player_id,year,r_total_caught_stealing,r_total_stolen_base,n_bolts,sprint_speed
count,133.000000,133.0,133.000000,133.000000,50.000000,133.000000
mean,633932.578947,2023.0,2.796992,11.751880,20.060000,27.395489
std,49936.834584,0.0,2.859750,12.585108,33.705453,1.242301
min,457759.000000,2023.0,0.000000,0.000000,1.000000,24.300000
25%,605204.000000,2023.0,1.000000,3.000000,2.000000,26.400000
50%,656305.000000,2023.0,2.000000,8.000000,6.000000,27.400000
75%,666969.000000,2023.0,4.000000,16.000000,18.000000,28.300000
max,807799.000000,2023.0,15.000000,73.000000,149.000000,30.500000


Below I checked for duplicates. I didn't think there would be any but you never know!

In [9]:
# Check for duplicated
len(player_data_df[player_data_df.duplicated()])

0

In [10]:
# Rewrite column names in title case
player_data_df.columns = player_data_df.columns.map(lambda x: x.title())

In [11]:
player_data_df

,"Last_Name, First_Name",Player_Id,Year,R_Total_Caught_Stealing,R_Total_Stolen_Base,N_Bolts,Sprint_Speed
0,"Candelario, Jeimer",600869,2023,1,8,NaN,27.5
1,"McMahon, Ryan",641857,2023,5,5,NaN,25.8
2,"Muncy, Max",571970,2023,2,1,NaN,26.9
3,"Soler, Jorge",624585,2023,0,1,NaN,26.6
4,"Edman, Tommy",669242,2023,4,27,7.0,28.8
...,...,...,...,...,...,...,...
128,"Benintendi, Andrew",643217,2023,2,13,NaN,27.3
129,"Machado, Manny",592518,2023,2,3,NaN,26.3
130,"Meneses, Joey",608841,2023,0,0,NaN,25.6
131,"Arcia, Orlando",606115,2023,0,1,NaN,25.9


In [ ]:
# Replace NaNs in N_Bolts with 0

# Exploratory Data Analysis

# Conclusions

## Limitations

## Recommendations

## Next Steps